In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager
matplotlib.font_manager._rebuild()
matplotlib.rcParams['font.family'] = ['Heiti TC']
import seaborn as sns
from scipy import stats
import numpy as np

ModuleNotFoundError: No module named 'pandas'

In [10]:
%reload_ext autoreload
%autoreload 3
%aimport Scripts

from Scripts.utils import util
from Scripts.utils import cleanText
from Scripts.dangerousDriving import bloodContentBin as baBin
from Scripts.dangerousDriving import keywordFeatures

In [3]:
pd.set_option('display.max_colwidth', 1000)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import linear_model
from sklearn.feature_selection import SelectFromModel
import jieba
import re

## Load Data

In [5]:
NAokCase= pd.read_csv('./dataframes/dangerousDriving/NAImprisonLessThanEq6.csv')

In [11]:
NAokCase.columns.values

array(['fact', 'lawArticle', 'accusation', 'moneyPunishment',
       'criminalName', 'deathPenalty', 'imprisonment', 'lifeImprisonment',
       'dateTime', 'province_CN', 'province_EN', 'city_CN', 'city_EN',
       'County_CN', 'bloodAlcoholContent', 'baNotes',
       'bloodContent10mgBin', 'bloodContent10mgBinNum'], dtype=object)

In [7]:
NAokCaseFact = NAokCase['fact'].tolist()

In [8]:
NAokCasesImpM = NAokCase['imprisonment'].tolist()

## Construct Feature Vector for each row

In [43]:
features = ['无证','超载','超速','相撞','无号牌','醉酒','碰撞','追尾','饮酒',
            '死亡','重伤','损失','事故','追尾','受损',
            '致人','恶劣','重大','逃逸','自首']

In [106]:
def genKeywordFeatureVecs(factList, bloodContentBinNumList):
    featureVecs = []
    for idx, fact in enumerate(factList):
        factVec = []
        for feature in features:
            if feature in fact:
                factVec.append(1)
            else:
                factVec.append(0)
#         factVec.append(bloodContentBinNumList[idx])
        featureVecs.append(factVec)
    return featureVecs

### National Overall

In [107]:
NAkwFtus = genKeywordFeatureVecs(NAokCaseFact, NAokCase['bloodContent10mgBinNum'].tolist())

In [92]:
NAkwFtus

[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 11],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 14],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12],
 [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 12],
 [0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 14],
 [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 12],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5

In [108]:
# NAokCase['keywordFeature'] = NAkwFtus
# NAokCase['keywordFeatureNoBA'] = NAkwFtus

In [110]:
# NAokCase.sample()

In [111]:
# NAokCase.to_csv('./dataframes/dangerousDriving/NAImprisonLessThanEq6KeywordFeatures.csv', index=False)

In [93]:
clf = linear_model.Lasso(alpha=1)

In [94]:
# lrmodel = clf.fit(NAkwFtus, NAokCasesImpM)

In [95]:
selector = SelectFromModel(estimator=clf).fit(NAkwFtus, NAokCasesImpM)


In [96]:
np.where(selector.get_support())

(array([20]),)

In [97]:
selector.estimator_.coef_

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        , -0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.        ,
        0.03348263])

### Xinjiang

In [98]:
XJokCases = NAokCase[NAokCase['province_EN'] == 'Xinjiang']
XJkwFtus = genKeywordFeatureVecs(XJokCases['fact'].tolist(), XJokCases['bloodContent10mgBinNum'].tolist())
XJy = XJokCases['imprisonment'].tolist()

In [99]:
len(XJkwFtus)

2659

In [100]:
clf = linear_model.Lasso(alpha=1)
# lrmodel = clf.fit(XJkwFtus, XJy)

In [101]:
selector = SelectFromModel(estimator=clf).fit(XJkwFtus, XJy)
np.where(selector.get_support())

(array([20]),)

In [102]:
selector.estimator_.coef_

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
       -0.        , -0.        ,  0.        ,  0.        , -0.        ,
        0.11295378])

In [103]:
selector.threshold_

1e-05